In [27]:
from flask import Flask
from flask_cors import CORS
from flask import request
import json
import pandas as pd
import pathlib
import heapq

import requests
import datetime
import base64
from urllib.parse import urlencode

from fastai.collab import *
from fastai.tabular.all import *

In [28]:
client_id = '7605835d7b0c4efd952bb9b511d824dc'
client_secret = '57cdae8d450c49f1b8b851f1fb4c31d7'

In [39]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception('You must set client_id and client_secret')
        client_creds = f'{client_id}:{client_secret}'
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            'Authorization': f'Basic {client_creds_b64}'
        }
    
    def get_token_data(self):
        return {
            'grant_type': 'client_credentials'
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            print(r)
            return False
        data = r.json()
        now = datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def is_token_expired(self):
        return self.access_token_expires < datetime.now()

In [40]:
client = SpotifyAPI(client_id, client_secret)
client.perform_auth()

True

In [78]:
path_playlists = r"D:\schule\diplomarbeit\converted_json\minimized_mpd_0-9999.json"
path_unique_tracks = r"D:\schule\diplomarbeit\converted_json\unique_tracks_0-9999.json"

mpd_unique_artists_file = r"D:\schule\diplomarbeit\converted_csv\unique_artists_0-9999.csv"
mpd_csv_file = r"D:\schule\diplomarbeit\converted_csv\mpd_slice_0-9999.csv"

path_models = Path("D:/schule/diplomarbeit/models")

In [79]:
save = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath   
learn = load_learner(path_models/'playlists_tracks_br_model_v1.pkl')
artist_learner = load_learner(path_models/'playlists_artists_br_model_v1.pkl')
pathlib.PosixPath = save

track_factors = learn.model.i_weight.weight
playlist_factors = learn.model.u_weight.weight

artist_factors = artist_learner.model.i_weight.weight

dls = learn.dls
artist_dls = artist_learner.dls

In [82]:
playlists_info = json.load(open(path_playlists, 'r'))
for playlist in playlists_info:
    del playlist['tracks']
playlists = json.load(open(path_playlists, 'r'))
unique_tracks = json.load(open(path_unique_tracks, 'r'))

artists = pd.read_csv(mpd_unique_artists_file, delimiter=',', encoding='utf-8', header=None,
                       names=['artist_uri','artist_name'], skiprows=1)

playlists_csv = pd.read_csv(mpd_csv_file, delimiter=',', encoding='utf-8', header=None, low_memory=False,
                       names=['pid','track_uri','rating','playlist_name','track_name','artist_uri','artist_name'], skiprows=1)

In [55]:
def sigmoid(x):
    return 2 * (1 / (1 + math.exp(-x)) - 0.5)

class Rating:
    def __init__(self, track_uri, track_name, artist_uri, artist_name, rating, pid):
        self.track_uri = track_uri
        self.track_name = track_name
        self.artist_uri = artist_uri
        self.artist_name = artist_name
        self.rating = rating
        self.pid = pid

    def __eq__(self, other):
        return self.track_uri == other.track_uri

    def __hash__(self):
        return hash(('track_uri', self.track_uri))

In [89]:
app = Flask("flowsy")
CORS(app)

@app.route('/token')
def getSpotifyToken():
    if(client.is_token_expired()):
        client.perform_auth()
    return json.dumps(client.access_token)

@app.route('/search/<filter>')
def search(filter):
    playlist_info = []
    for playlist in playlists_info:
        if(filter in playlist['name']):
            playlist_info.append(playlist)
    return json.dumps(playlist_info)

@app.route('/playlist/<pid>')
def getPlaylistInfo(pid):
    return playlists[int(pid)]

@app.route('/tracks/<pid>')
def getTracksOfPlaylist(pid):
    return json.dumps(playlists[int(pid)]['tracks'])

@app.route('/recommendationv2/<pid>')
def getRecommendationv2(pid):
    
    # Get all artists of playlist
    
    artist_counts = {}
    artist_uris = list(playlists_csv.loc[playlists_csv['pid'] == int(pid)]['artist_uri'])
    for artist_uri in artist_uris:
        artist_counts[artist_uri] = artist_uris.count(artist_uri)
        
    # Get similar artists of playlist
    
    similar_artists = []
    artist_uris = list(artist_counts.keys())
    for artist_uri in artist_uris:
        idx = artist_dls.classes['artist_uri'].o2i[artist_uri]
        distances = nn.CosineSimilarity(dim=1)(artist_factors, artist_factors[idx][None])
        idxs = distances.argsort(descending=True)[1:4]
        similar_artist_uris = list(artist_dls.classes['artist_uri'][idxs])
        similar_artists.extend(similar_artist_uris)

    similar_artists_filtered = [similar_artist for similar_artist in similar_artists if similar_artist not in artist_uris]
    
    # Get similar playlists 
    playlist_factors = learn.model.u_weight.weight
    idx = dls.classes['pid'].o2i[int(pid)]
    distances = nn.CosineSimilarity(dim=1)(playlist_factors, playlist_factors[idx][None])
    idxs = distances.argsort(descending=True)[1:100]
    pids = list(dls.classes['pid'][idxs])
    
    # Append all tracks of similar playlist to list and a basic rating   
    ratings = []
    count = 1
    for p in pids:
        track_uris = list(playlists_csv.loc[playlists_csv['pid'] == p]['track_uri'])
        track_names = list(playlists_csv.loc[playlists_csv['pid'] == p]['track_name'])
        artist_uris = list(playlists_csv.loc[playlists_csv['pid'] == p]['artist_uri'])
        artist_names = list(playlists_csv.loc[playlists_csv['pid'] == p]['artist_name'])
        for track_uri, track_name, artist_uri, artist_name in zip(track_uris, track_names, artist_uris, artist_names):
            rating = Rating(track_uri, track_name, artist_uri, artist_name, count, pid)
            ratings.append(rating)
        if(count > 0):
            count -= 0.05
            
    # Improve rating for tracks of same artists
    for rating in ratings:
        if rating.artist_uri in artist_counts:
            rating.rating = rating.rating + sigmoid(artist_counts[rating.artist_uri])
            
    # Improve rating for tracks of similar artists   
    for rating in ratings:
        if rating.artist_uri in similar_artists_filtered:
            rating.rating = rating.rating + 1
    
    # Remove same tracks   
    unique_ratings = list(set(ratings))

    # Remove recommended tracks that are in the playlist
    tracks_of_playlist = list(playlists_csv.loc[playlists_csv['pid'] == int(pid)]['track_uri'])
    filtered_ratings = [unique_rating for unique_rating in unique_ratings if not unique_rating.track_uri in tracks_of_playlist]
            
    #return json.dumps(list([unique_rating.track_name for unique_rating in unique_ratings]))
    # Sort ratings
    filtered_ratings.sort(key=lambda x: x.rating, reverse=True)
    
    # Limit same artists in recommendation
    top_track_uris = []
    for filtered_rating in filtered_ratings:
        count = sum(rating.artist_uri == filtered_rating.artist_uri for rating in filtered_ratings)
        if(count < 2):
            top_track_uris.append(filtered_rating.track_uri)

    # Translate top tracks
    top_tracks = []
    for track_uri in top_track_uris[:20]:
        for track in unique_tracks:
            if(track['track_uri'] == track_uri):
                top_tracks.append(track)
                break
    return json.dumps(top_tracks)

@app.route('/recommendation/<pid>')
def getRecommendation(pid):
    idx = dls.classes['pid'].o2i[int(pid)]
    distances = np.array(list((playlist_factors[idx] @ track_factor for track_factor in track_factors)))
    n = int(request.args.get('n'))
    temp = n + playlists[int(pid)]['num_tracks']
    indices = heapq.nlargest(temp, range(len(distances)), distances.take)
    tracks_of_playlist = list(track['track_uri'] for track in playlists[int(pid)]['tracks'])
    top_track_uris = list((dls.classes['track_uri'][idx] for idx in indices))
    filtered_top_track_uris = [top_track_uri for top_track_uri in top_track_uris if not top_track_uri in tracks_of_playlist]
    top_tracks = []
    for track_uri in filtered_top_track_uris[:n]:
        for track in unique_tracks:
            if(track['track_uri'] == track_uri):
                top_tracks.append(track)
                break
    return json.dumps(top_tracks)
                
    
app.run(host='0.0.0.0')

 * Serving Flask app "flowsy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Nov/2020 17:38:21] "GET /recommendationv2/3 HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2020 17:38:43] "GET /recommendationv2/255 HTTP/1.1" 200 -
10.0.0.4 - - [20/Nov/2020 17:39:02] "GET /search/electronic HTTP/1.1" 200 -
10.0.0.4 - - [20/Nov/2020 17:39:04] "GET /playlist/7722 HTTP/1.1" 200 -
10.0.0.4 - - [20/Nov/2020 17:39:04] "GET /tracks/7722 HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2020 17:39:23] "GET /recommendationv2/7722 HTTP/1.1" 200 -
